# Preprocess - Get data train
This notebook is used to collect data for training purposes. The data in this notebook will collect data from different sources. The goal is a Xy-dataset in parquet format, with:
- The number of WMO-clients is the target-variable y. 
- Index contains a region and time interval
- Filename contains a datetime suffix

## Content
* **Imports**: Imports of needed Python packages
* **Settings**: Hard coded variables needed to collect data like sources, tablenames, columnnames, etc. 
* **Funtions**: Resuable functions
* **Load data from sources**: Seperate paragraph for each source
    * CBS: WMO_TABLES
    * CBS: WIJK_TABLES
* **Combine multiple sources**: Combining all data to one table
* **Write result**: Writing result to '../data'
* **Appendix**: Usefull code to preserve
    * Code examples to get a subset of the DataFrame based on multiindex
    * Code to get current versions of loaded packages

## Requirements
The packages to be installed (besides standard Python packages) are:
* pandas >=1.1.5
* cbsodata >=1.3.3

# Imports

In [1]:
import pandas as pd
import cbsodata
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Settings

In [2]:
# URL of the CBS Statline database
CBS_OPEN_URL = 'opendata.cbs.nl'
# CBS tables with information of the WMO clients
WMO_TABLES = {'2020': '84907NED',
            '2019': '84664NED',
            '2018': '84421NED',
            '2017': '83818NED',
            '2016': '83620NED',
            '2015': '83267NED'}
# CBS tables with the information of key figures for a neighbourhood
WIJK_TABLES = {'2020': '84799NED', 
               '2019': '84583NED', 
               '2018': '84286NED', 
               '2017': '83765NED', 
               '2016': '83487NED', 
               '2015': '83220NED'} 
# Dictionary with columns that need to be renamed to avoid duplicates for neighbourhood data
DOUBLETROUBLECOLNAMES_WIJK = {'GemiddeldElektriciteitsverbruikTotaal_47': 'GemiddeldElektriciteitsverbruikTotaal_47',
                             'Appartement_48': 'GemElectriciteitsverbruikAppartement_48',
                             'Tussenwoning_49': 'GemElectriciteitsverbruikTussenwoning_49',
                             'Hoekwoning_50': 'GemElectriciteitsverbruikHoekwoning_50',
                             'TweeOnderEenKapWoning_51': 'GemElectriciteitsverbruikTweeOnderEenKapWoning_51',
                             'VrijstaandeWoning_52': 'GemElectriciteitsverbruikVrijstaandeWoning_52',
                             'Huurwoning_53': 'GemElectriciteitsverbruikHuurwoning_53',
                             'EigenWoning_54': 'GemElectriciteitsverbruikEigenWoning_54',
                              'Koopwoning_54' : 'GemElectriciteitsverbruikEigenWoning_54',
                             'GemiddeldAardgasverbruikTotaal_55': 'GemiddeldAardgasverbruikTotaal_55',
                             'Appartement_56': 'GemGasverbruikAppartement_56',
                             'Tussenwoning_57': 'GemGasverbruikTussenwoning_57',
                             'Hoekwoning_58': 'GemGasverbruikHoekwoning_58',
                             'TweeOnderEenKapWoning_59': 'GemGasverbruikTweeOnderEenKapWoning_59',
                             'VrijstaandeWoning_60': 'GemGasverbruikVrijstaandeWoning_60',
                             'Huurwoning_61': 'GemGasverbruikHuurwoning_61',
                             'EigenWoning_62': 'GemGasverbruikEigenWoning_62',
                              'Koopwoning_62': 'GemGasverbruikEigenWoning_62',
                             'PercentageWoningenMetStadsverwarming_63': 'PercentageWoningenMetStadsverwarming_63'}

# Functions

In [3]:
def get_and_combine_cbs_tables(dict_tables, double_trouble_colnames=None, url='opendata.cbs.nl'):
    """
    Method to get multiple simular tables in the CBS database.
    
    :params dict(str: str) tables: Dictionary with as key the period and as value the table name
    :params dict(str: str) double_trouble_colnames: Dictionary with columnnames that will cause trouble if the suffix is deleted
    :params str url: URL of the catalog of the CBS databases, i.e.: 'opendata.cbs.nl'
    
    return: pd.DataFrame
    """

    print(f"Number of tables to collect: {len(dict_tables)}")
    
    df= pd.DataFrame()
    for interval, table in dict_tables.items():
        print(f"Pythonic iteration {interval} for table {table}")
        try:
            df_sub = pd.DataFrame(cbsodata.get_data(table, catalog_url=url))
            if double_trouble_colnames:
                df_sub = df_sub.rename(columns=double_trouble_colnames)
            cols_wijk_stripped = [i.rstrip('0123456789').replace("_", "").lower() for i in list(df_sub.columns)]
            dict_wijk_cols_renamed = {key: value for key, value in zip(iter(df_sub.columns), iter(cols_wijk_stripped))}
            df_sub = df_sub.rename(columns=dict_wijk_cols_renamed)
            df_sub['interval'] = interval
            # print(list(df_sub.columns))
        except Exception:
            df_sub = pd.DataFrame()
            pass
        df = pd.concat([df, df_sub])
        # print(list(df.columns))
    return df

def rename_and_subset_cols(df, dict_rename, list_cols, include=True):
    """
    Method to rename and subset certain columns from a DataFrame. 
    
    :params pd.DataFrame df: DataFrame with several columns
    :params dict(str:str) dict_rename: Dictionary with a dictionary where the keys are the original columnnames
                                       and the values are the new column names
    :params list(str) list_cols: List of columns to keep/drop
    :params bool include: Boolean value to indicate if the columns from list_cols should be kept or dropped. Default 'true' to keep.
    
    return: pd.DataFrame
    """
    
    df = df.rename(columns=dict_rename)
    if include:
        df = df[list_cols]
    else:
        df = df.drop(list_cols, axis=1)
    
    return df

def get_wmo_subtable(df, region=None, period=None, form=None):
    """
    Method to subset the dataframe with WMO data.
    
    :params pd.DataFrame df: DataFrame with the WMO data from CBS with the columns: 
                            [['codering_regio', 'perioden', 'financieringsvorm', 'wmoclienten', 'wmoclientenper1000inwoners']]
    :params str region: String to choose region. Possible strings are: "gemeente", "wijk"
    :params str period: String to choose period. Possible strings are: "jaar", "halfjaar" 
    :params str form: String to choose form of financing. Possible strings are: "totaal", "per type"
    
    return: pd.DataFrame
    """
    
    if region == "gemeente":
        df = df[df.codering_regio.str.startswith('GM', na=False)]
    if region == "wijk":
        df = df[df.codering_regio.str.startswith('WK', na=False)]
    if region == "buurt":
        df = df[df.codering_regio.str.startswith('BU', na=False)]
    if period == "jaar":
        df = df[~df.perioden.str.contains("halfjaar", na=False)]
    if period == "halfjaar":
        df = df[df.perioden.str.contains("halfjaar", na=False)]
    if form == "totaal":
        df = df[df.financieringsvorm == "Totaal"]
    if form == "per type":
        df = df[df.financieringsvorm != "Totaal"]
    
    return df

def downcast_variables_dataframe(df):
    """
    Method to downcast the variables in a DataFrame
    
    :params pd.DataFrame: df: DataFrame to downcast
    
    return: pd.DataFrame
    """
    df_downy = df.copy()
    # Downcast dataset
    df_downy[df_downy.select_dtypes(include='object').columns] = df_downy.select_dtypes(include='object').astype('category')

    for old, new in [('integer', 'unsigned'), ('float', 'float')]:
        for col in df.select_dtypes(include=old).columns:
            df_downy.loc[:,col] = pd.to_numeric(df_downy.loc[:,col], downcast=new)
    return df_downy

# Load data from sources

## CBS: WMO_TABLES

In [4]:
%%time
# Get WMO
df_wmo = get_and_combine_cbs_tables(dict_tables=WMO_TABLES, url=CBS_OPEN_URL)
df_wmo_sub = rename_and_subset_cols(df=df_wmo, 
                                    dict_rename={"codering": "codering_regio"}, 
                                    list_cols=['interval', 'codering_regio', 'perioden', 'financieringsvorm', 'wmoclienten', 'wmoclientenper1000inwoners'])

df_wmo_total = get_wmo_subtable(df=df_wmo_sub, region="wijk", period="jaar", form="totaal")
df_wmo_total = downcast_variables_dataframe(df_wmo_total)
df_wmo_total = df_wmo_total.set_index(['codering_regio', 'perioden'])
df_wmo_total = df_wmo_total.drop(['interval'], axis=1)
df_wmo_total.sample(5)

Number of tables to collect: 6
Pythonic iteration 2020 for table 84907NED
Pythonic iteration 2019 for table 84664NED
Pythonic iteration 2018 for table 84421NED
Pythonic iteration 2017 for table 83818NED
Pythonic iteration 2016 for table 83620NED
Pythonic iteration 2015 for table 83267NED
Wall time: 44.2 s


## CBS: WIJK_TABLES

In [5]:
%%time
# Get Wijkdata
df_wijk = get_and_combine_cbs_tables(dict_tables=WIJK_TABLES, double_trouble_colnames = DOUBLETROUBLECOLNAMES_WIJK, url=CBS_OPEN_URL)
DICT_WIJK_COLS_RENAMED = {'codering':'codering_regio', 
                          'interval':'perioden'}
df_wijk_sub = rename_and_subset_cols(df=df_wijk, 
                                     dict_rename=DICT_WIJK_COLS_RENAMED, 
                                     list_cols=['id', 'wijkenenbuurten', 'soortregio', 'indelingswijzigingwijkenenbuurten'], 
                                     include=False)
df_wijk_sub['codering_regio'] = df_wijk_sub['codering_regio'].str.strip()
df_wijk_sub['gemeentenaam'] = df_wijk_sub['gemeentenaam'].str.strip()
df_wijk_total = df_wijk_sub[df_wijk_sub.codering_regio.str.startswith('WK', na=False)]
df_wijk_total = downcast_variables_dataframe(df_wijk_total)
df_wijk_total = df_wijk_total.set_index(['codering_regio', 'perioden'])
df_wijk_total.sample(5)

Number of tables to collect: 6
Pythonic iteration 2020 for table 84799NED
Pythonic iteration 2019 for table 84583NED
Pythonic iteration 2018 for table 84286NED
Pythonic iteration 2017 for table 83765NED
Pythonic iteration 2016 for table 83487NED
Pythonic iteration 2015 for table 83220NED
Wall time: 2min 50s


## Source: Type of information

In [6]:
# Possible other source (to be added later):

# Combine multiple sources

In [7]:
df_dataset_WMO = pd.merge(df_wmo_total, df_wijk_total, how='inner', left_index=True, right_index=True)
df_dataset_WMO.sample(5)

,,financieringsvorm,wmoclienten,wmoclientenper1000inwoners,gemeentenaam,aantalinwoners,mannen,vrouwen,k0tot15jaar,k15tot25jaar,k25tot45jaar,k45tot65jaar,k65jaarofouder,ongehuwd,gehuwd,gescheiden,verweduwd,westerstotaal,nietwesterstotaal,marokko,nederlandseantillenenaruba,suriname,turkije,overignietwesters,geboortetotaal,geboorterelatief,sterftetotaal,sterfterelatief,huishoudenstotaal,eenpersoonshuishoudens,huishoudenszonderkinderen,huishoudensmetkinderen,gemiddeldehuishoudensgrootte,bevolkingsdichtheid,woningvoorraad,gemiddeldewoningwaarde,percentageeengezinswoning,percentagemeergezinswoning,percentagebewoond,percentageonbewoond,koopwoningen,huurwoningentotaal,inbezitwoningcorporatie,inbezitoverigeverhuurders,eigendomonbekend,bouwjaarvoor2000,bouwjaarvanaf2000,gemiddeldelektriciteitsverbruiktotaal,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruikeigenwoning,gemiddeldaardgasverbruiktotaal,gemgasverbruikappartement,gemgasverbruiktussenwoning,gemgasverbruikhoekwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgasverbruikhuurwoning,gemgasverbruikeigenwoning,percentagewoningenmetstadsverwarming,aantalinkomensontvangers,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,k40personenmetlaagsteinkomen,k20personenmethoogsteinkomen,actieven1575jaar,k40huishoudensmetlaagsteinkomen,k20huishoudensmethoogsteinkomen,huishoudensmeteenlaaginkomen,huishonderofrondsociaalminimum,personenpersoortuitkeringbijstand,personenpersoortuitkeringao,personenpersoortuitkeringww,personenpersoortuitkeringaow,bedrijfsvestigingentotaal,alandbouwbosbouwenvisserij,bfnijverheidenenergie,gihandelenhoreca,hjvervoerinformatieencommunicatie,klfinancieledienstenonroerendgoed,mnzakelijkedienstverlening,rucultuurrecreatieoverigediensten,personenautostotaal,personenautosbrandstofbenzine,personenautosoverigebrandstof,personenautosperhuishouden,personenautosnaaroppervlakte,motorfietsen,afstandtothuisartsenpraktijk,afstandtotgrotesupermarkt,afstandtotkinderdagverblijf,afstandtotschool,scholenbinnen3km,oppervlaktetotaal,oppervlakteland,oppervlaktewater,meestvoorkomendepostcode,dekkingspercentage,matevanstedelijkheid,omgevingsadressendichtheid,totaaldiefstaluitwoningschuured,vernielingmisdrijftegenopenbareorde,geweldsenseksuelemisdrijven,personenautosjongerdan6jaar,personenautos6jaarenouder,bedrijfsmotorvoertuigen
codering_regio,perioden,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
WK011421,2018,Totaal,65.0,51.0,Emmen,1275,670,605,185,170,250,415,240,545,575,75,75,80,10,0,5,0,0,5,5,5,10,9,525,145,175,205,2.4,72.0,532,186.0,94.0,6.0,95.0,5.0,79.0,21.0,12.0,9.0,0.0,89.0,11.0,3620.0,2120.0,2190.0,2450.0,3640.0,3990.0,2540.0,3830.0,1990.0,1230.0,1380.0,1360.0,1810.0,2280.0,1620.0,2070.0,NaN,1000,NaN,NaN,42.3,15.8,62.2,30.0,21.4,4.3,4.3,10.0,50.0,30.0,230.0,105.0,30.0,25.0,15.0,5.0,5.0,10.0,15.0,730,580,150,1.4,41.0,75,1.2,3.6,1.3,1.3,1.2,1769,1764,5,.,NaN,5.0,84.0,1.0,1.0,4.0,NaN,NaN,NaN
WK194041,2019,Totaal,250.0,75.0,De Fryske Marren,3340,1645,1695,530,435,690,930,760,1615,1290,175,245,160,60,0,0,5,0,55,25,8,50,16,1385,465,430,490,2.3,304.0,1441,212.0,87.0,13.0,97.0,3.0,66.0,34.0,24.0,10.0,0.0,89.0,11.0,2790.0,1340.0,2450.0,2450.0,2650.0,3560.0,2010.0,3250.0,1370.0,670.0,1200.0,1190.0,1320.0,1730.0,1090.0,1540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,170.0,30.0,710.0,365.0,30.0,110.0,70.0,20.0,30.0,75.0,30.0,1655,1170,485,1.2,151.0,140,0.8,0.7,0.7,0.6,3.0,1189,1099,90,.,NaN,5.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN
WK078600,2018,Totaal,560.0,65.0,Grave,8680,4335,4345,1345,900,1940,2600,1885,3810,3570,770,530,855,415,25,20,50,20,290,65,8,85,10,3885,1315,1280,1295,2.2,2992.0,3982,186.0,79.0,21.0,97.0,3.0,63.0,36.0,26.0,10.0,0.0,90.0,10.0,2910.0,2130.0,2980.0,3060.0,3

In [8]:
df_dataset_WMO.shape

(11267, 109)

# Write result

In [9]:
suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

df_dataset_WMO.to_parquet(f'../data/df_get_for_train_WMO_{suffix_datetime}.parquet.gzip',
              compression='gzip')

# Appendix
## Code examples to get a subset of the DataFrame based on multiindex

In [10]:
# Subset on columnvalue:
df_dataset_WMO[df_dataset_WMO['gemeentenaam']=='Nijmegen']

# One row / record
df_dataset_WMO.loc[('WK026801', '2019')]

# Multiple rows / records based on combination of the multiindex
df_dataset_WMO.loc[[('WK026801', '2018'), ('WK026802', '2018')]]

# Multiple rows / records for one column (works only for series)
df_dataset_WMO['gemeentenaam'].loc[(['WK026801', 'WK026802'], ['2018', '2019'])]

# Multiple rows based on both indexes:
df_dataset_WMO.loc(axis=0)[['WK026801', 'WK026802'], ['2018', '2019']]

# Subset with IndexSlice
idx = pd.IndexSlice
# Subset on one of the multiindex and select a column
df_dataset_WMO.loc[idx[:, ['2018', '2019']], idx["gemeentenaam"]]
# Subset on one of the multiindex and select all columns
df_dataset_WMO.loc[idx['WK026801', :], idx[:]]

financieringsvorm  wmoclienten  wmoclientenper1000inwoners gemeentenaam  aantalinwoners  mannen  vrouwen  k0tot15jaar  k15tot25jaar  k25tot45jaar  k45tot65jaar  k65jaarofouder  ongehuwd  gehuwd  gescheiden  verweduwd  westerstotaal  nietwesterstotaal  marokko  nederlandseantillenenaruba  suriname  turkije  overignietwesters geboortetotaal geboorterelatief sterftetotaal sterfterelatief  huishoudenstotaal  eenpersoonshuishoudens  huishoudenszonderkinderen  huishoudensmetkinderen  gemiddeldehuishoudensgrootte  bevolkingsdichtheid woningvoorraad gemiddeldewoningwaarde percentageeengezinswoning percentagemeergezinswoning percentagebewoond percentageonbewoond koopwoningen huurwoningentotaal inbezitwoningcorporatie inbezitoverigeverhuurders eigendomonbekend bouwjaarvoor2000 bouwjaarvanaf2000 gemiddeldelektriciteitsverbruiktotaal gemelectriciteitsverbruikappartement gemelectriciteitsverbruiktussenwoning gemelectriciteitsverbruikhoekwoning  \
codering_regio perioden                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
WK026801       2019                Totaal        610.0                        51.0     Nijmegen           11850    5865     5985          320          3915          4610          1850            1165      9510    1305         790        240           1755               1075       90                         110        75      125                685             50                4            40               3               9020                    6915                       1650                     450                           1.3               9523.0           5480                  222.0                      12.0                       88.0              94.0                 6.0         23.0               76.0                    31.0                      45.0              1.0             84.0              16.0                                2070.0                               1950.0                                2860.0                              2810.0   
               2018                Totaal        585.0                        49.0     Nijmegen           12055    5925     6130          440          4090          4510          1895            1135      9640    1370         800        235           1670               1245      100                         110        75      125                845             40                3            40               3               9060                    6980                       1620                     450                           1.3               9685.0           5427                  198.0                      12.0                       88.0              93.0                 7.0         22.0               76.0                    32.0                      43.0              2.0             84.0              16.0                                2110.0                               2000.0                                2840.0                              2910.0   
               2017                Totaal        550.0                        47.0     Nijmegen           11650    5755     5895        

In [11]:
df_dataset_WMO[df_dataset_WMO['gemeentenaam']=='Nijmegen']

,,financieringsvorm,wmoclienten,wmoclientenper1000inwoners,gemeentenaam,aantalinwoners,mannen,vrouwen,k0tot15jaar,k15tot25jaar,k25tot45jaar,k45tot65jaar,k65jaarofouder,ongehuwd,gehuwd,gescheiden,verweduwd,westerstotaal,nietwesterstotaal,marokko,nederlandseantillenenaruba,suriname,turkije,overignietwesters,geboortetotaal,geboorterelatief,sterftetotaal,sterfterelatief,huishoudenstotaal,eenpersoonshuishoudens,huishoudenszonderkinderen,huishoudensmetkinderen,gemiddeldehuishoudensgrootte,bevolkingsdichtheid,woningvoorraad,gemiddeldewoningwaarde,percentageeengezinswoning,percentagemeergezinswoning,percentagebewoond,percentageonbewoond,koopwoningen,huurwoningentotaal,inbezitwoningcorporatie,inbezitoverigeverhuurders,eigendomonbekend,bouwjaarvoor2000,bouwjaarvanaf2000,gemiddeldelektriciteitsverbruiktotaal,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruikeigenwoning,gemiddeldaardgasverbruiktotaal,gemgasverbruikappartement,gemgasverbruiktussenwoning,gemgasverbruikhoekwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgasverbruikhuurwoning,gemgasverbruikeigenwoning,percentagewoningenmetstadsverwarming,aantalinkomensontvangers,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,k40personenmetlaagsteinkomen,k20personenmethoogsteinkomen,actieven1575jaar,k40huishoudensmetlaagsteinkomen,k20huishoudensmethoogsteinkomen,huishoudensmeteenlaaginkomen,huishonderofrondsociaalminimum,personenpersoortuitkeringbijstand,personenpersoortuitkeringao,personenpersoortuitkeringww,personenpersoortuitkeringaow,bedrijfsvestigingentotaal,alandbouwbosbouwenvisserij,bfnijverheidenenergie,gihandelenhoreca,hjvervoerinformatieencommunicatie,klfinancieledienstenonroerendgoed,mnzakelijkedienstverlening,rucultuurrecreatieoverigediensten,personenautostotaal,personenautosbrandstofbenzine,personenautosoverigebrandstof,personenautosperhuishouden,personenautosnaaroppervlakte,motorfietsen,afstandtothuisartsenpraktijk,afstandtotgrotesupermarkt,afstandtotkinderdagverblijf,afstandtotschool,scholenbinnen3km,oppervlaktetotaal,oppervlakteland,oppervlaktewater,meestvoorkomendepostcode,dekkingspercentage,matevanstedelijkheid,omgevingsadressendichtheid,totaaldiefstaluitwoningschuured,vernielingmisdrijftegenopenbareorde,geweldsenseksuelemisdrijven,personenautosjongerdan6jaar,personenautos6jaarenouder,bedrijfsmotorvoertuigen
codering_regio,perioden,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
WK026801,2019,Totaal,610.0,51.0,Nijmegen,11850,5865,5985,320,3915,4610,1850,1165,9510,1305,790,240,1755,1075,90,110,75,125,685,50,4,40,3,9020,6915,1650,450,1.3,9523.0,5480,222.0,12.0,88.0,94.0,6.0,23.0,76.0,31.0,45.0,1.0,84.0,16.0,2070.0,1950.0,2860.0,2810.0,2710.0,NaN,1990.0,2280.0,990.0,920.0,1420.0,1610.0,1750.0,NaN,980.0,1010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,410.0,160.0,1060.0,2025.0,0.0,105.0,775.0,165.0,170.0,485.0,325.0,3465,2660,805,0.4,2786.0,330,0.5,0.3,0.5,0.6,15.1,142,124,18,.,NaN,1.0,3759.0,NaN,NaN,NaN,NaN,NaN,NaN
WK026802,2019,Totaal,1865.0,54.0,Nijmegen,34355,16070,18285,3080,8790,8885,7905,5705,23255,7570,2250,1270,4415,2480,195,245,200,415,1435,170,5,300,9,21890,14765,4165,2955,1.5,4623.0,14954,318.0,40.0,60.0,96.0,4.0,47.0,53.0,33.0,20.0,1.0,87.0,13.0,2470.0,2070.0,2740.0,3020.0,3490.0,4690.0,2130.0,2860.0,1290.0,980.0,1440.0,1730.0,2120.0,3230.0,1060.0,1560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.0,1170.0,460.0,5290.0,3045.0,0.0,195.0,415.0,275.0,340.0,1225.0,590.0,11835,9575,2255,0.5,1592.0,740,0.6,0.6,0.4,0.5,13.9,771,743,28,.,NaN,1.0,2982.0,NaN,NaN,NaN,NaN,NaN,NaN
WK026803,2019,Totaal,1110.0,72.0,Nijmegen,15405,7355,8050,1900,2575,5290,3690,1965,9805,3705,1355,530,1675,2490,460,180,100,825,935,195,13,110,7,8730,4790,2040,1900,1.7,8498.0,

## Code to get current versions of loaded packages

In [12]:
print('\n'.join(f'{m.__name__} {m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

pandas 1.1.5
cbsodata 1.3.3
